In [18]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode

from typing import Annotated, TypedDict, List

from langchain_community.tools import TavilySearchResults
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

import sqlite3

load_dotenv()
groq_model = os.getenv("groq_model")

class BaseStateClass(TypedDict):
    messages: Annotated[List, add_messages]

sqlite_conn = sqlite3.connect('approval_review_tool_call.sqlite', check_same_thread=False)
memory = SqliteSaver(sqlite_conn)

search_tools = TavilySearchResults(max_results = 2)
tool = [search_tools]

llm = ChatGroq(model = groq_model)
llm_with_tools = llm.bind_tools(tools= tool)


def model(state: BaseStateClass):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

tools = ToolNode(tools = tool)  

def tools_router(state: BaseStateClass):
    last_message = state["messages"][-1]

    if(hasattr(last_message, "tool_calls") and last_message.tool_calls):
        return "tools"
    else:
        return END

  
graph = StateGraph(BaseStateClass)

graph.add_node("model", model)
graph.add_node("tools", tools)
graph.set_entry_point("model")
graph.add_conditional_edges("model", tools_router)
graph.add_edge("tools", "model")
app = graph.compile(checkpointer= memory, interrupt_after=["tools"])



In [19]:

config = {"configurable": {"thread_id": "1"}}

events = app.stream({
    "messages": [HumanMessage(content="What is the current wether in lalitpur")]
}, config= config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()

print("next state", app.get_state(config).next)

================================ Human Message =================================

What is the current wether in lalitpur
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (ckd8c4q1d)
 Call ID: ckd8c4q1d
  Args:
    query: current weather in Lalitpur June 17, 2025
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Lalitpur weather in June 2025 - Meteoprog.com", "url": "https://www.meteoprog.com/weather/Lalitpurin/month/june/", "content": "Lalitpur (India) weather in June 2025 ☀️ Accurate weather forecast for Lalitpur ... 17 Jun. +38°+29°. 18 Jun. +36°+28°. 19 Jun. +37°+27°. 20 Jun. +38°+29°. 21 Jun.", "score": 0.9293445}, {"title": "Lalitpur, Central Development Region - weather outlook for 17 June ...", "url": "https://wisemeteo.com/en/country/nepal/region/central-development-region/city/jitpur/date/06-17", "content": "Lalitpur, Central Dev

In [20]:
events = app.stream(None, config, stream_mode="values")
for event in events:
    event["messages"][-1].pretty_print()


================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Lalitpur weather in June 2025 - Meteoprog.com", "url": "https://www.meteoprog.com/weather/Lalitpurin/month/june/", "content": "Lalitpur (India) weather in June 2025 ☀️ Accurate weather forecast for Lalitpur ... 17 Jun. +38°+29°. 18 Jun. +36°+28°. 19 Jun. +37°+27°. 20 Jun. +38°+29°. 21 Jun.", "score": 0.9293445}, {"title": "Lalitpur, Central Development Region - weather outlook for 17 June ...", "url": "https://wisemeteo.com/en/country/nepal/region/central-development-region/city/jitpur/date/06-17", "content": "Lalitpur, Central Development Region - weather forecast for 17 June 2025 · 1 mph SE. Cloud cover (points). 70. 6:00. Rain. 70°. Probability of precipitation. 70%.", "score": 0.91760606}]
================================== Ai Message ==================================

The current weather in Lalitpur on June 17, 2025 is expected to be sunny with a high tem

In [ ]:
from pprint import pprint
events = app.stream(None, config, stream_mode="values")
snapshot = app.get_state(config)
print("current state", snapshot.values)
for event in events:
    pprint(event.get("messages", "NO messages found"))

current state StateSnapshot(values={'messages': [HumanMessage(content='What is the current wether in lalitpur', additional_kwargs={}, response_metadata={}, id='4a79b5e6-6731-40fc-b0e7-61525e9eb372'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '3c4ssf6g8', 'function': {'arguments': '{"query":"current weather in Lalitpur"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 289, 'total_tokens': 312, 'completion_time': 0.030666667, 'prompt_time': 0.017128794, 'queue_time': 0.051909306, 'total_time': 0.047795461}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7735026b-75d4-4d25-a228-78bbe8e7100d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Lalitpur'}, 'id': '3c4ssf6g8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 289, 'output_tokens': 2